# Adaptive Cutoff Validation on 2M Odlyzko Zeros

## Context

The mollified Dirichlet polynomial $S_w(T)$ uses a cutoff $X = T^{\theta(T)}$.

Two functional forms for $\theta(T)$ are tested:

1. **Polynomial**: $\theta(T) = a - b/\log T + c/\log^2 T$ — truncated expansion
2. **Shifted-log**: $\theta(T) = a - b/(\log T + d)$ — resums the full $1/\log T$ series

The shifted-log form captures all higher-order Euler product corrections through
the geometric series $b/(\log T + d) = b/\log T - bd/\log^2 T + bd^2/\log^3 T - \cdots$

**Key discovery**: The formula $\theta(T) = \frac{7}{6} - \frac{\varphi}{\log T - 2}$
uses only GIFT topological constants ($\dim K_7$, $N_\text{gen}$, $p_2$, $\varphi$) with zero free parameters.

## Validation Tests

1. **Phase 1**: Full 2M evaluation with all primes up to 500k
2. **T5a**: Beat 200 random constant-$\theta$ baselines
3. **T5b**: Beat 200 random adaptive baselines (polynomial + shifted-log)
4. **T7**: Bootstrap 95% CI contains $\alpha = 1$
5. **T8**: No significant alpha drift ($p > 0.05$)
6. **Visualizations**: Window alphas, bootstrap histogram, form comparison
7. **Final scorecard** with validation summary

In [ ]:
# ================================================================
# COLAB SETUP — Mount Google Drive
# Skip this cell if running locally
# ================================================================
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted at /content/drive")
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("Not running in Colab — using local filesystem")

In [ ]:
import numpy as np
from scipy.special import loggamma, lambertw
from scipy import stats
import matplotlib.pyplot as plt
import json, time, os, math

# ================================================================
# CONSTANTS
# ================================================================
phi = (1 + math.sqrt(5)) / 2          # Golden ratio φ ≈ 1.61803
EULER_GAMMA = 0.5772156649015329       # Euler-Mascheroni γ

# ================================================================
# GPU DETECTION
# ================================================================
os.environ.setdefault('CUDA_PATH', '/usr')  # Ubuntu nvrtc path

try:
    import cupy as cp
    _ = cp.maximum(cp.array([1.0]), 0.0)  # test kernel compilation
    GPU = True
    mem_free = cp.cuda.Device(0).mem_info[0] // 1024 // 1024
    print(f"[GPU] CuPy {cp.__version__}, free={mem_free} MB")
    # Adaptive batch sizes based on VRAM
    if mem_free > 20_000:      # A100 / V100
        PRIME_BATCH = 1000
        ZERO_CHUNK = 500_000
    elif mem_free > 6_000:     # T4 / RTX 3060+
        PRIME_BATCH = 500
        ZERO_CHUNK = 200_000
    else:                      # RTX 2050 / small GPUs
        PRIME_BATCH = 200
        ZERO_CHUNK = 100_000
except (ImportError, Exception) as e:
    GPU = False
    PRIME_BATCH = 200
    ZERO_CHUNK = 200_000
    print(f"[CPU] No GPU ({e}), using NumPy — expect ~40 min/candidate")

# ================================================================
# CONFIGURATION
# ================================================================
ZEROS_LOCAL = 'outputs/riemann_zeros_2M_genuine.npy'
ZEROS_DRIVE = '/content/drive/MyDrive/riemann_zeros_2M_genuine.npy'
ZEROS_FILE = ZEROS_LOCAL if os.path.exists(ZEROS_LOCAL) else ZEROS_DRIVE

P_MAX = 500_000          # Full prime sieve
K_MAX = 3                # Prime power cutoff
N_BOOTSTRAP = 2000       # Bootstrap resamples for T7
N_RANDOM_CONST = 200     # Random constant baselines for T5a
N_RANDOM_3D = 200        # Random 3-param baselines for T5b

# Window setup for drift analysis
N_WINDOWS = 6

# Output
OUTPUT_DIR = 'outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"\nConfiguration: P_MAX={P_MAX}, K_MAX={K_MAX}")
print(f"  Zeros file: {ZEROS_FILE}")
print(f"  GPU batching: prime_batch={PRIME_BATCH}, zero_chunk={ZERO_CHUNK:,}")
mode = "GPU float64 batched" if GPU else "CPU float64 batched"
print(f"  Mode: {mode}")
print(f"  Constants: phi={phi:.5f}, gamma={EULER_GAMMA:.7f}")

## 1. Load Zeros & Compute Infrastructure

In [ ]:
# ================================================================
# LOAD ZEROS
# ================================================================
all_zeros = np.load(ZEROS_FILE)
N_TOTAL = len(all_zeros)
gamma_n = all_zeros
print(f"Loaded {N_TOTAL:,} zeros, T range: [{gamma_n[0]:.2f}, {gamma_n[-1]:.2f}]")

# ================================================================
# RIEMANN-SIEGEL THETA
# ================================================================
def theta_rs(t):
    """Riemann-Siegel theta function."""
    t = np.asarray(t, dtype=np.float64)
    return np.imag(loggamma(0.25 + 0.5j * t)) - 0.5 * t * np.log(np.pi)

def theta_deriv(t):
    """theta'(t) ~ 0.5 * log(t / (2*pi))."""
    return 0.5 * np.log(np.maximum(np.asarray(t, dtype=np.float64), 1.0) / (2 * np.pi))

# ================================================================
# SMOOTH ZEROS (Newton iteration)
# ================================================================
def smooth_zeros(N):
    """Compute gamma0_n where theta_RS(gamma0_n) = (n - 1.5) * pi."""
    ns = np.arange(1, N + 1, dtype=np.float64)
    targets = (ns - 1.5) * np.pi
    w = np.real(lambertw(ns / np.e))
    t = np.maximum(2 * np.pi * ns / w, 2.0)
    for it in range(40):
        dt = (theta_rs(t) - targets) / np.maximum(np.abs(theta_deriv(t)), 1e-15)
        t -= dt
        if np.max(np.abs(dt)) < 1e-12:
            print(f"  Newton converged in {it+1} iterations")
            break
    return t

print("Computing smooth zeros for all 2M zeros...")
t0 = time.time()
gamma0 = smooth_zeros(N_TOTAL)
delta = gamma_n - gamma0
tp = theta_deriv(gamma0)
print(f"  Done in {time.time()-t0:.1f}s")
print(f"  delta: mean={delta.mean():.6f}, std={delta.std():.6f}")

# ================================================================
# PRIME SIEVE
# ================================================================
def sieve(N):
    is_p = np.ones(N + 1, dtype=bool); is_p[:2] = False
    for i in range(2, int(N**0.5) + 1):
        if is_p[i]: is_p[i*i::i] = False
    return np.where(is_p)[0]

t0 = time.time()
primes = sieve(P_MAX)
print(f"Sieved {len(primes):,} primes up to {P_MAX:,} [{time.time()-t0:.1f}s]")

## 2. Core Functions

In [ ]:
# ================================================================
# COSINE-SQUARED KERNEL
# ================================================================
def w_cosine(x):
    """Cosine-squared mollifier: cos^2(pi*x/2) for x < 1, 0 otherwise."""
    return np.where(x < 1.0, np.cos(np.pi * x / 2)**2, 0.0)

# ================================================================
# MOLLIFIED PRIME SUM — GPU-ACCELERATED
#
# Two modes:
#   POLYNOMIAL:  theta(T) = a - b/logT + c/log^2 T        (d_shift=0)
#   SHIFTED-LOG: theta(T) = a - b/(logT + d_shift)         (c=0, d_shift!=0)
#
# Strategy: chunk over zeros, batch over primes.
# ================================================================
def prime_sum_var(g0, tp_v, primes, k_max, theta_inf, theta_coeff, c_coeff=0.0,
                  d_shift=0.0, chunk_size=None, batch_size=None, verbose=False):
    """
    Mollified prime sum with chunked zeros + batched primes.
    
    Args:
        theta_inf: asymptotic theta (= a)
        theta_coeff: leading correction (= -b)
        c_coeff: quadratic correction for polynomial mode (= c)
        d_shift: log shift for shifted-log mode (= d in a - b/(logT + d))
    
    When d_shift != 0: theta = theta_inf + theta_coeff / (logT + d_shift)
    When d_shift == 0: theta = theta_inf + theta_coeff/logT + c_coeff/log^2 T
    """
    if chunk_size is None:
        chunk_size = ZERO_CHUNK
    if batch_size is None:
        batch_size = PRIME_BATCH
    
    xp = cp if GPU else np
    N = len(g0)
    result = np.zeros(N, dtype=np.float64)
    log_primes_np = np.log(primes.astype(np.float64))
    shifted_mode = (d_shift != 0.0)
    
    n_chunks = (N + chunk_size - 1) // chunk_size
    t_start = time.time()
    
    for ic in range(n_chunks):
        lo = ic * chunk_size
        hi = min(lo + chunk_size, N)
        N_c = hi - lo
        
        # Move chunk to device
        g0_c = xp.asarray(g0[lo:hi], dtype=xp.float64)
        tp_c = xp.asarray(tp_v[lo:hi], dtype=xp.float64)
        log_g0 = xp.log(xp.maximum(g0_c, 2.0))
        
        if shifted_mode:
            # SHIFTED-LOG: theta = a - b/(logT + d)
            # theta_coeff = -b, so theta = theta_inf + theta_coeff/(logT + d_shift)
            denom = log_g0 + xp.float64(d_shift)
            denom = xp.maximum(denom, 0.1)  # safety: avoid division by zero
            theta_per = xp.float64(theta_inf) + xp.float64(theta_coeff) / denom
            theta_per = xp.clip(theta_per, 0.5, 2.0)
            log_X = theta_per * log_g0
        elif theta_coeff == 0.0 and c_coeff == 0.0:
            log_X = xp.float64(theta_inf) * log_g0
        else:
            # POLYNOMIAL: theta = a + coeff/logT + c/log^2 T
            theta_per = xp.float64(theta_inf) + xp.float64(theta_coeff) / log_g0
            if c_coeff != 0.0:
                theta_per = theta_per + xp.float64(c_coeff) / log_g0**2
            theta_per = xp.clip(theta_per, 0.5, 2.0)
            log_X = theta_per * log_g0
        
        S = xp.zeros(N_c, dtype=xp.float64)
        log_X_max = float(xp.max(log_X))
        
        for m in range(1, k_max + 1):
            cutoff = log_X_max / m
            j_max = int(np.searchsorted(log_primes_np, cutoff + 0.1))
            if j_max == 0:
                continue
            
            for b_start in range(0, j_max, batch_size):
                b_end = min(b_start + batch_size, j_max)
                
                logp_b = xp.asarray(log_primes_np[b_start:b_end], dtype=xp.float64)
                p_b = xp.asarray(primes[b_start:b_end].astype(np.float64))
                
                x = (xp.float64(m) * logp_b[:, None]) / log_X[None, :]
                w = xp.where(x < 1.0, xp.cos(xp.float64(math.pi / 2) * x)**2,
                             xp.float64(0))
                phase = g0_c[None, :] * (xp.float64(m) * logp_b[:, None])
                coeff = xp.float64(1.0 / m) / p_b ** (m / 2.0)
                S -= xp.sum(w * xp.sin(phase) * coeff[:, None], axis=0)
                del x, w, phase, coeff
        
        chunk_result = -S / tp_c
        
        if GPU:
            result[lo:hi] = cp.asnumpy(chunk_result)
            cp.get_default_memory_pool().free_all_blocks()
        else:
            result[lo:hi] = chunk_result
        
        if verbose and n_chunks > 1:
            elapsed = time.time() - t_start
            pct = (ic + 1) / n_chunks
            eta = elapsed / pct * (1 - pct) if pct > 0 else 0
            print(f"      [{hi:>10,}/{N:>10,}] {pct*100:5.1f}%  "
                  f"[{elapsed/60:.1f}m, ETA {eta/60:.1f}m]", flush=True)
    
    return result

# ================================================================
# METRICS (unchanged)
# ================================================================
def compute_alpha_R2(delta, delta_pred):
    denom = np.dot(delta_pred, delta_pred)
    alpha = float(np.dot(delta, delta_pred) / denom) if denom > 0 else 0.0
    residual = delta - delta_pred
    R2 = float(1.0 - np.var(residual) / np.var(delta))
    residual_scaled = delta - alpha * delta_pred
    R2_scaled = float(1.0 - np.var(residual_scaled) / np.var(delta))
    return alpha, R2, R2_scaled

def compute_localization(delta, delta_pred, gamma_n):
    half_gaps = np.diff(gamma_n) / 2.0
    residual = delta - delta_pred
    n = min(len(residual) - 1, len(half_gaps))
    localized = np.abs(residual[1:n+1]) < half_gaps[:n]
    return float(np.mean(localized))

def compute_window_alphas(delta, delta_pred, n_windows=6):
    N = len(delta)
    bounds = [int(i * N / n_windows) for i in range(n_windows + 1)]
    alphas = []
    for i in range(n_windows):
        lo, hi = bounds[i], bounds[i + 1]
        d_w = delta[lo:hi]
        dp_w = delta_pred[lo:hi]
        denom = np.dot(dp_w, dp_w)
        alpha = float(np.dot(d_w, dp_w) / denom) if denom > 0 else 0.0
        alphas.append(alpha)
    return alphas, bounds

def compute_drift(alphas):
    if len(alphas) < 3:
        return 0.0, 1.0
    x = np.arange(len(alphas))
    slope, intercept, r, p, se = stats.linregress(x, alphas)
    return slope, p

def bootstrap_alpha(delta, delta_pred, n_boot=2000, ci=0.95, seed=42):
    rng = np.random.default_rng(seed)
    N = len(delta)
    alphas = np.empty(n_boot)
    for i in range(n_boot):
        idx = rng.integers(0, N, size=N)
        d = delta[idx]
        dp = delta_pred[idx]
        denom = np.dot(dp, dp)
        alphas[i] = np.dot(d, dp) / denom if denom > 0 else 0.0
    lo = np.percentile(alphas, (1 - ci) / 2 * 100)
    hi = np.percentile(alphas, (1 + ci) / 2 * 100)
    return alphas, lo, hi

def clopper_pearson_ci(k, n, alpha=0.05):
    from scipy.stats import beta as beta_dist
    lo = beta_dist.ppf(alpha / 2, k, n - k + 1) if k > 0 else 0.0
    hi = beta_dist.ppf(1 - alpha / 2, k + 1, n - k) if k < n else 1.0
    return lo, hi

print("Core functions defined (polynomial + shifted-log modes).")
print(f"Benchmarking on 5k zeros, 669 primes...")
_g0 = gamma0[:5000]; _tp = tp[:5000]; _pr = primes[:669]
t0 = time.time()
_ = prime_sum_var(_g0, _tp, _pr, K_MAX, 7/6, -phi, d_shift=-2)
t_bench = time.time() - t0
print(f"  Shifted-log: {t_bench*1000:.0f} ms")
t0 = time.time()
_ = prime_sum_var(_g0, _tp, _pr, K_MAX, 7/6, -(math.e/phi), -2*phi)
t_poly = time.time() - t0
print(f"  Polynomial:  {t_poly*1000:.0f} ms")

## 3. Define Candidate Models

Two functional forms are tested:

**Polynomial**: $\theta(T) = a - b/\log T + c/\log^2 T$ — truncated expansion

**Shifted-log**: $\theta(T) = a - b/(\log T + d)$ — resums the full $1/\log T$ series

The shifted-log form arises from the geometric series:
$$\frac{b}{\log T + d} = \frac{b}{\log T} - \frac{bd}{\log^2 T} + \frac{bd^2}{\log^3 T} - \cdots$$

**Key discovery**: $\theta(T) = \frac{7}{6} - \frac{\varphi}{\log T - 2}$ uses only GIFT topological constants:
- $7/6 = \dim(K_7) / (2 \cdot N_\text{gen})$
- $\varphi$ = golden ratio (G$_2$ metric eigenvalue)
- $2 = p_2$ = Pontryagin class contribution

In [ ]:
# ================================================================
# CANDIDATE MODELS
# Format: (name, a, b, c, d_shift)
#
# POLYNOMIAL mode (d_shift=0): theta = a - b/logT + c/log^2 T
# SHIFTED-LOG mode (d_shift!=0): theta = a - b/(logT + d_shift)
#
# prime_sum_var convention: theta_coeff = -b, c_coeff = c
# ================================================================
CANDIDATES = [
    # --- Shifted-log: θ = a − b/(logT + d) ---
    # GIFT pure: all constants from topology (dim_K7, N_gen, p2, phi)
    ("SL: 7/6 - phi/(logT - 2)",
     7/6, phi, 0.0, -2.0),

    # Polynomial expansion of GIFT pure: -phi*(-2)^0/logT = -phi/logT, -phi*(-2)/log^2T = 2phi/log^2T
    # This is the polynomial winner from 3D exploration — included to compare forms
    ("SL: 7/6 - (e/phi)/(logT - 2phi^2/e)",
     7/6, math.e/phi, 0.0, -2*phi**2/math.e),

    # Euler-Mascheroni shift
    ("SL: 7/6 - phi/(logT - gamma)",
     7/6, phi, 0.0, -EULER_GAMMA),

    # Alternative a: 13/11 (close to 7/6)
    ("SL: 13/11 - phi/(logT - 2)",
     13/11, phi, 0.0, -2.0),

    # --- Polynomial: θ = a − b/logT + c/log²T ---
    # Winner from 3D exploration (α=0.9998 on 2M)
    ("POLY: 7/6 - e/phi/logT - 2phi/log2T",
     7/6, math.e/phi, -2*phi, 0.0),

    # Second-best polynomial
    ("POLY: 7/6 - 13/8/logT - 7/2/log2T",
     7/6, 13/8, -7/2, 0.0),

    # Third polynomial
    ("POLY: 13/11 - 7/4/logT - 2phi/log2T",
     13/11, 7/4, -2*phi, 0.0),

    # --- 2D baselines (no subleading correction) ---
    ("[2D] 5/4 - e/logT",
     5/4, math.e, 0.0, 0.0),

    ("[2D] 11/9 - 5/2/logT",
     11/9, 5/2, 0.0, 0.0),
]

print(f"Candidates to validate: {len(CANDIDATES)}")
print()
for i, (name, a, b, c, d) in enumerate(CANDIDATES):
    if d != 0:
        print(f"  [{i+1:2d}] {name}")
        print(f"       theta = {a:.6f} - {b:.6f}/(logT + ({d:.6f}))")
    elif c != 0:
        print(f"  [{i+1:2d}] {name}")
        print(f"       theta = {a:.6f} - {b:.6f}/logT + ({c:.6f})/log^2 T")
    else:
        print(f"  [{i+1:2d}] {name}")
        print(f"       theta = {a:.6f} - {b:.6f}/logT")

## 4. Phase 1 — Full 2M Zero Evaluation

Run each candidate on all 2,001,052 zeros with all primes up to 500k.
Both polynomial and shifted-log modes are supported transparently.

In [ ]:
# ================================================================
# FULL 2M EVALUATION — with checkpointing
# ================================================================
CHECKPOINT_FILE = os.path.join(OUTPUT_DIR, 'theta_validation_checkpoint.json')

# Try to resume from checkpoint
phase1_results = []
start_idx = 0

if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE) as f:
        checkpoint = json.load(f)
    # Verify checkpoint matches current candidates
    if len(checkpoint) > 0 and checkpoint[0].get('name') == CANDIDATES[0][0]:
        for cr in checkpoint:
            cr['delta_pred'] = None  # Will recompute if needed
            phase1_results.append(cr)
        start_idx = len(phase1_results)
        print(f"Resumed from checkpoint: {start_idx}/{len(CANDIDATES)} candidates done")
    else:
        print("Checkpoint from different candidate list — starting fresh")
else:
    print("No checkpoint found, starting fresh")

print("=" * 80)
print(f"PHASE 1: Full 2M evaluation ({N_TOTAL:,} zeros, {len(primes):,} primes)")
print("=" * 80)

t_phase1 = time.time()

for i in range(start_idx, len(CANDIDATES)):
    name, a, b, c, d_shift = CANDIDATES[i]
    is_shifted = (d_shift != 0)
    mode_str = "shifted-log" if is_shifted else ("3-param" if c != 0 else "2-param")
    
    print(f"\n  [{i+1}/{len(CANDIDATES)}] {name}  ({mode_str})")
    if is_shifted:
        print(f"    a={a:.8f}  b={b:.8f}  d={d_shift:.8f}")
    else:
        print(f"    a={a:.8f}  b={b:.8f}  c={c:.8f}")
    
    t0 = time.time()
    
    # Compute mollified sum (theta_coeff = -b)
    delta_pred = prime_sum_var(gamma0, tp, primes, K_MAX,
                               theta_inf=a, theta_coeff=-b,
                               c_coeff=c, d_shift=d_shift,
                               verbose=True)
    
    elapsed = time.time() - t0
    
    # Metrics
    alpha, R2, R2_sc = compute_alpha_R2(delta, delta_pred)
    loc = compute_localization(delta, delta_pred, gamma_n)
    alphas_w, bounds_w = compute_window_alphas(delta, delta_pred, N_WINDOWS)
    drift_slope, drift_p = compute_drift(alphas_w)
    
    # Check monotonicity
    diffs = np.diff(alphas_w)
    is_monotone = all(d <= 0 for d in diffs) or all(d >= 0 for d in diffs)
    
    r = {
        'name': name,
        'a': float(a), 'b': float(b), 'c': float(c), 'd_shift': float(d_shift),
        'mode': 'shifted-log' if is_shifted else 'polynomial',
        'alpha': float(alpha),
        'abs_alpha_minus_1': float(abs(alpha - 1)),
        'R2': float(R2),
        'R2_scaled': float(R2_sc),
        'localization': float(loc),
        'drift_slope': float(drift_slope),
        'drift_p': float(drift_p),
        'window_alphas': [float(x) for x in alphas_w],
        'is_monotone': bool(is_monotone),
        'delta_pred': delta_pred,  # Keep for later phases
        'elapsed_s': float(elapsed),
    }
    phase1_results.append(r)
    
    print(f"    alpha  = {alpha:+.6f}  |alpha-1| = {abs(alpha-1):.6f}")
    print(f"    R2     = {R2:.6f}  R2_sc = {R2_sc:.6f}  loc = {loc:.4f}")
    print(f"    drift  = {drift_slope:+.6f}  (p={drift_p:.4f})  {'monotone' if is_monotone else 'non-monotone'}")
    print(f"    windows = {[f'{x:.4f}' for x in alphas_w]}")
    print(f"    [{elapsed/60:.1f}m]")
    
    # --- CHECKPOINT: save after each candidate ---
    save_cp = [{k: v for k, v in r.items() if k != 'delta_pred'}
               for r in phase1_results]
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(save_cp, f, indent=2)
    print(f"    [checkpoint saved: {len(phase1_results)}/{len(CANDIDATES)}]")

total_p1 = time.time() - t_phase1
print(f"\nPhase 1 complete in {total_p1/3600:.1f}h")

# Rank by |alpha - 1|
phase1_ranked = sorted(phase1_results, key=lambda r: r['abs_alpha_minus_1'])

# Only recompute delta_pred for top N candidates needed by T7/T8
N_TOP_RECOMPUTE = min(5, len(phase1_ranked))
for r in phase1_ranked[:N_TOP_RECOMPUTE]:
    if r['delta_pred'] is None:
        # Find matching candidate
        idx = next(j for j, (_, a, b, c, d) in enumerate(CANDIDATES)
                   if abs(a - r['a']) < 1e-10 and abs(b - r['b']) < 1e-10
                   and abs(c - r['c']) < 1e-10 and abs(d - r['d_shift']) < 1e-10)
        name, a, b, c, d_shift = CANDIDATES[idx]
        print(f"  Recomputing delta_pred for top candidate: {name}...")
        r['delta_pred'] = prime_sum_var(gamma0, tp, primes, K_MAX,
                                         theta_inf=a, theta_coeff=-b,
                                         c_coeff=c, d_shift=d_shift,
                                         verbose=True)

print("\n" + "=" * 80)
print("RANKING by |alpha - 1|:")
print("=" * 80)
print(f"{'Rk':<4} {'Name':<40} {'alpha':>10} {'|a-1|':>10} {'drift':>10} {'p':>8} {'mode':>10}")
print("-" * 92)
for rk, r in enumerate(phase1_ranked, 1):
    print(f"{rk:<4} {r['name']:<40} {r['alpha']:>+10.6f} {r['abs_alpha_minus_1']:>10.6f} "
          f"{r['drift_slope']:>+10.6f} {r['drift_p']:>8.4f} {r['mode']:>10}")

## 5. Phase 2 — Monte Carlo Validation (T5)

### T5a: Beat 200 random constant-$\theta$ baselines
For each random draw, pick $\theta \sim \text{Uniform}(0.6, 1.5)$ (constant, no correction).
Compute $R^2$ on 2M zeros. The candidate must have higher $R^2$ than all 200.

### T5b: Beat 200 random 3-parameter baselines
For each random draw, pick $(a, b, c) \sim$ Uniform on reasonable ranges.
The candidate must have higher $R^2$ than all 200.

Both tests use a **subset** of zeros for speed (every 10th zero = 200k).

In [ ]:
# ================================================================
# T5 MONTE CARLO BASELINES
# Use every 10th zero for speed (200k zeros)
# ================================================================
STRIDE_MC = 10
idx_mc = np.arange(0, N_TOTAL, STRIDE_MC)
g0_mc = gamma0[idx_mc]
tp_mc = tp[idx_mc]
delta_mc = delta[idx_mc]
gn_mc = gamma_n[idx_mc]
N_mc = len(idx_mc)

# Primes subset for MC (use first 5000 primes for speed)
P_MC = min(5000, len(primes))
primes_mc = primes[:P_MC]

print(f"Monte Carlo setup: {N_mc:,} zeros (stride {STRIDE_MC}), {P_MC:,} primes")

# Compute R^2 for the best candidate on MC subset
best = phase1_ranked[0]
print(f"\nBest candidate for T5: {best['name']} ({best['mode']})")

# Recompute on MC subset for fair comparison
dp_best_mc = prime_sum_var(g0_mc, tp_mc, primes_mc, K_MAX,
                           theta_inf=best['a'], theta_coeff=-best['b'],
                           c_coeff=best['c'], d_shift=best['d_shift'])
_, R2_best_mc, _ = compute_alpha_R2(delta_mc, dp_best_mc)
print(f"  R^2 (MC subset) = {R2_best_mc:.6f}")

# ================================================================
# T5a: Random constant-theta baselines
# ================================================================
print(f"\n--- T5a: {N_RANDOM_CONST} random constant-theta baselines ---")
rng = np.random.default_rng(2026)
R2_const = np.empty(N_RANDOM_CONST)
t0 = time.time()

for k in range(N_RANDOM_CONST):
    theta_rand = rng.uniform(0.6, 1.5)
    dp_rand = prime_sum_var(g0_mc, tp_mc, primes_mc, K_MAX,
                            theta_rand, 0.0, 0.0)
    _, R2_rand, _ = compute_alpha_R2(delta_mc, dp_rand)
    R2_const[k] = R2_rand
    if (k + 1) % 50 == 0:
        print(f"  {k+1}/{N_RANDOM_CONST} [{time.time()-t0:.0f}s]")

n_beat_const = int(np.sum(R2_best_mc > R2_const))
T5a_pass = bool(n_beat_const == N_RANDOM_CONST)
T5a_margin = float(R2_best_mc - np.max(R2_const))
cp_lo_a, cp_hi_a = clopper_pearson_ci(n_beat_const, N_RANDOM_CONST)
print(f"  Beat {n_beat_const}/{N_RANDOM_CONST} random constant baselines")
print(f"  Best random R^2: {np.max(R2_const):.6f}")
print(f"  Candidate R^2:   {R2_best_mc:.6f}")
print(f"  Margin: {T5a_margin:+.6f}")
print(f"  Clopper-Pearson 95% CI for p(beat): [{cp_lo_a:.4f}, {cp_hi_a:.4f}]")
print(f"  T5a: {'PASS' if T5a_pass else 'FAIL'}")

# ================================================================
# T5b: Random 3-parameter baselines (polynomial + shifted-log)
# ================================================================
print(f"\n--- T5b: {N_RANDOM_3D} random adaptive baselines ---")
R2_3d = np.empty(N_RANDOM_3D)
t0 = time.time()

for k in range(N_RANDOM_3D):
    a_rand = rng.uniform(1.0, 1.5)
    b_rand = rng.uniform(0.5, 6.0)
    # Mix polynomial and shifted-log randomly (50/50)
    if rng.random() < 0.5:
        c_rand = rng.uniform(-10.0, 10.0)
        dp_rand = prime_sum_var(g0_mc, tp_mc, primes_mc, K_MAX,
                                a_rand, -b_rand, c_rand, d_shift=0.0)
    else:
        d_rand = rng.uniform(-5.0, 2.0)
        dp_rand = prime_sum_var(g0_mc, tp_mc, primes_mc, K_MAX,
                                a_rand, -b_rand, 0.0, d_shift=d_rand)
    _, R2_rand, _ = compute_alpha_R2(delta_mc, dp_rand)
    R2_3d[k] = R2_rand
    if (k + 1) % 50 == 0:
        print(f"  {k+1}/{N_RANDOM_3D} [{time.time()-t0:.0f}s]")

n_beat_3d = int(np.sum(R2_best_mc > R2_3d))
T5b_pass = bool(n_beat_3d == N_RANDOM_3D)
T5b_margin = float(R2_best_mc - np.max(R2_3d))
cp_lo_b, cp_hi_b = clopper_pearson_ci(n_beat_3d, N_RANDOM_3D)
print(f"  Beat {n_beat_3d}/{N_RANDOM_3D} random adaptive baselines")
print(f"  Best random R^2: {np.max(R2_3d):.6f}")
print(f"  Candidate R^2:   {R2_best_mc:.6f}")
print(f"  Margin: {T5b_margin:+.6f}")
print(f"  Clopper-Pearson 95% CI for p(beat): [{cp_lo_b:.4f}, {cp_hi_b:.4f}]")
print(f"  T5b: {'PASS' if T5b_pass else 'FAIL'}")

T5_pass = T5a_pass and T5b_pass
print(f"\n  T5 overall: {'PASS' if T5_pass else 'FAIL'}")

# LEE correction note
print(f"\n  Note: With {len(CANDIDATES)} candidates tested, the")
print(f"  look-elsewhere effect (LEE) should be considered.")
print(f"  Effective p-value after Bonferroni: p < {0.05/len(CANDIDATES):.4f}")

## 6. Phase 3 — Bootstrap Confidence Interval (T7)

Test: does the 95% bootstrap CI for $\alpha$ contain 1.0?

Applied to the **top candidates** on the full 2M dataset.

In [ ]:
# ================================================================
# T7: BOOTSTRAP CI for top candidates
# ================================================================
N_TOP = min(5, len(phase1_ranked))
print(f"T7 Bootstrap CI ({N_BOOTSTRAP} resamples) on top {N_TOP} candidates")
print("=" * 70)

t7_results = {}

for i in range(N_TOP):
    r = phase1_ranked[i]
    name = r['name']
    dp = r['delta_pred']
    
    print(f"\n  [{i+1}/{N_TOP}] {name}")
    t0 = time.time()
    
    boot_alphas, ci_lo, ci_hi = bootstrap_alpha(delta, dp, 
                                                n_boot=N_BOOTSTRAP, seed=42+i)
    
    contains_one = bool(ci_lo <= 1.0 <= ci_hi)
    elapsed = time.time() - t0
    
    t7_results[name] = {
        'boot_alphas': boot_alphas,
        'ci_lo': float(ci_lo),
        'ci_hi': float(ci_hi),
        'T7_pass': contains_one,
    }
    
    print(f"    alpha = {r['alpha']:+.6f}")
    print(f"    95% CI: [{ci_lo:.6f}, {ci_hi:.6f}]")
    print(f"    Contains 1.0: {'YES -> PASS' if contains_one else 'NO -> FAIL'}")
    print(f"    [{elapsed:.1f}s]")

# Update phase1 results with T7
for r in phase1_results:
    if r['name'] in t7_results:
        r['T7_pass'] = t7_results[r['name']]['T7_pass']
        r['T7_ci_lo'] = t7_results[r['name']]['ci_lo']
        r['T7_ci_hi'] = t7_results[r['name']]['ci_hi']
    else:
        r['T7_pass'] = None

## 7. Phase 4 — Drift Analysis (T8)

Test: is the alpha drift slope statistically insignificant ($p > 0.05$)?

The alpha drift measures how well the correction scales across the full $T$ range.
A persistent downward drift suggests the model degrades at high $T$.

In [ ]:
# ================================================================
# T8: DRIFT ANALYSIS
# Extended window analysis with more granular windows
# ================================================================
N_DRIFT_WINDOWS = 12  # More windows for better drift resolution

print(f"T8 Drift Analysis ({N_DRIFT_WINDOWS} windows) on top {N_TOP} candidates")
print("=" * 70)

t8_results = {}

for i in range(N_TOP):
    r = phase1_ranked[i]
    name = r['name']
    dp = r['delta_pred']
    
    # Compute with more windows for finer resolution
    alphas_fine, bounds_fine = compute_window_alphas(delta, dp, N_DRIFT_WINDOWS)
    drift_slope_fine, drift_p_fine = compute_drift(alphas_fine)
    
    # T-range of each window midpoint
    T_mids = []
    for j in range(N_DRIFT_WINDOWS):
        mid_idx = (bounds_fine[j] + bounds_fine[j+1]) // 2
        T_mids.append(float(gamma0[mid_idx]))
    
    T8_pass = bool(drift_p_fine > 0.05)
    
    t8_results[name] = {
        'alphas_fine': [float(x) for x in alphas_fine],
        'T_mids': T_mids,
        'drift_slope': float(drift_slope_fine),
        'drift_p': float(drift_p_fine),
        'T8_pass': T8_pass,
    }
    
    print(f"\n  [{i+1}/{N_TOP}] {name}")
    print(f"    drift slope = {drift_slope_fine:+.6f}  (p = {drift_p_fine:.4f})")
    print(f"    T8: {'PASS' if T8_pass else 'FAIL'}")
    print(f"    alpha range: [{min(alphas_fine):.4f}, {max(alphas_fine):.4f}]")
    print(f"    T range: [{T_mids[0]:.0f}, {T_mids[-1]:.0f}]")

# Update phase1 results with T8
for r in phase1_results:
    if r['name'] in t8_results:
        r['T8_pass'] = t8_results[r['name']]['T8_pass']
        r['T8_drift_slope_fine'] = t8_results[r['name']]['drift_slope']
        r['T8_drift_p_fine'] = t8_results[r['name']]['drift_p']
    else:
        r['T8_pass'] = None

## 8. Visualizations

In [ ]:
# ================================================================
# MULTI-PANEL FIGURE
# ================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('3D Adaptive Cutoff Validation — 2M Odlyzko Zeros', fontsize=14, fontweight='bold')

# Colors for candidates
colors = plt.cm.tab10(np.linspace(0, 1, N_TOP))

# --- Panel 1: Window alphas for top candidates ---
ax = axes[0, 0]
ax.set_title('Window Alpha vs Window Index')
for i in range(N_TOP):
    r = phase1_ranked[i]
    name = r['name']
    if name in t8_results:
        alphas_f = t8_results[name]['alphas_fine']
        label = f"{name} (a={r['alpha']:.4f})"
        ax.plot(range(len(alphas_f)), alphas_f, 'o-', color=colors[i],
                label=label, markersize=4, linewidth=1.2)
ax.axhline(y=1.0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
ax.set_xlabel('Window index')
ax.set_ylabel(r'$\alpha$')
ax.legend(fontsize=7, loc='best')
ax.grid(True, alpha=0.3)

# --- Panel 2: Window alphas vs log(T) ---
ax = axes[0, 1]
ax.set_title(r'Window Alpha vs $\log T$')
for i in range(N_TOP):
    r = phase1_ranked[i]
    name = r['name']
    if name in t8_results:
        T_m = t8_results[name]['T_mids']
        alphas_f = t8_results[name]['alphas_fine']
        ax.plot(np.log(T_m), alphas_f, 'o-', color=colors[i],
                label=name, markersize=4, linewidth=1.2)
ax.axhline(y=1.0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
ax.set_xlabel(r'$\log T$')
ax.set_ylabel(r'$\alpha$')
ax.legend(fontsize=7, loc='best')
ax.grid(True, alpha=0.3)

# --- Panel 3: Bootstrap histogram for best candidate ---
ax = axes[1, 0]
best_name = phase1_ranked[0]['name']
if best_name in t7_results:
    boot = t7_results[best_name]
    ax.hist(boot['boot_alphas'], bins=50, density=True, alpha=0.7, color='steelblue',
            edgecolor='white', linewidth=0.5)
    ax.axvline(x=1.0, color='red', linestyle='--', linewidth=1.5, label=r'$\alpha = 1$')
    ax.axvline(x=boot['ci_lo'], color='orange', linestyle=':', linewidth=1.2, label=f'95% CI')
    ax.axvline(x=boot['ci_hi'], color='orange', linestyle=':', linewidth=1.2)
    ax.axvline(x=phase1_ranked[0]['alpha'], color='green', linestyle='-', linewidth=1.5,
               label=f"OLS alpha={phase1_ranked[0]['alpha']:.4f}")
    ax.set_title(f'Bootstrap Distribution — {best_name}')
    ax.legend(fontsize=8)
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('Density')
ax.grid(True, alpha=0.3)

# --- Panel 4: |alpha-1| comparison bar chart ---
ax = axes[1, 1]
names_short = [r['name'][:25] for r in phase1_ranked]
abs_a1 = [r['abs_alpha_minus_1'] for r in phase1_ranked]
bar_colors = ['green' if r.get('T7_pass') else 'tomato' for r in phase1_ranked]
ax.barh(range(len(phase1_ranked)), abs_a1, color=bar_colors, alpha=0.8)
ax.set_yticks(range(len(phase1_ranked)))
ax.set_yticklabels(names_short, fontsize=8)
ax.set_xlabel(r'$|\alpha - 1|$')
ax.set_title(r'$|\alpha - 1|$ Ranking (green = T7 pass)')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
fig_path = os.path.join(OUTPUT_DIR, 'theta_3d_validation.png')
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_path}")
plt.show()

## 9. Shifted-log vs Polynomial vs 2D Comparison

Direct comparison of the best shifted-log, best polynomial, and best 2D candidate.
Shows whether the shifted-log resummation improves over the truncated polynomial.

In [ ]:
# ================================================================
# SHIFTED-LOG vs POLYNOMIAL vs 2D COMPARISON
# ================================================================
results_sl = [r for r in phase1_results if r['mode'] == 'shifted-log']
results_poly = [r for r in phase1_results if r['mode'] == 'polynomial' and r['c'] != 0]
results_2d = [r for r in phase1_results if r['mode'] == 'polynomial' and r['c'] == 0]

best_sl = min(results_sl, key=lambda r: r['abs_alpha_minus_1']) if results_sl else None
best_poly = min(results_poly, key=lambda r: r['abs_alpha_minus_1']) if results_poly else None
best_2d = min(results_2d, key=lambda r: r['abs_alpha_minus_1']) if results_2d else None

print("=" * 70)
print("SHIFTED-LOG vs POLYNOMIAL vs 2D COMPARISON")
print("=" * 70)

for label, best in [("Best shifted-log", best_sl), ("Best polynomial", best_poly), ("Best 2D", best_2d)]:
    if best:
        print(f"\n{label}: {best['name']}")
        print(f"  alpha = {best['alpha']:+.6f}  |alpha-1| = {best['abs_alpha_minus_1']:.6f}")
        print(f"  R2    = {best['R2']:.6f}  drift = {best['drift_slope']:+.6f} (p={best['drift_p']:.4f})")
        print(f"  T7: {best.get('T7_pass', '?')}  T8: {best.get('T8_pass', '?')}")

# Improvement ratios
if best_sl and best_poly and best_poly['abs_alpha_minus_1'] > 0:
    ratio = best_poly['abs_alpha_minus_1'] / best_sl['abs_alpha_minus_1'] if best_sl['abs_alpha_minus_1'] > 0 else float('inf')
    print(f"\n  Shifted-log vs polynomial: {ratio:.1f}x closer to alpha=1")
if best_sl and best_2d and best_2d['abs_alpha_minus_1'] > 0:
    ratio2d = best_2d['abs_alpha_minus_1'] / best_sl['abs_alpha_minus_1'] if best_sl['abs_alpha_minus_1'] > 0 else float('inf')
    print(f"  Shifted-log vs 2D:         {ratio2d:.1f}x closer to alpha=1")

# Window-by-window comparison figure
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plot_items = []
if best_2d:
    plot_items.append((best_2d, 'gray', 's-', '2D'))
if best_poly:
    plot_items.append((best_poly, 'steelblue', 'D-', 'Poly'))
if best_sl:
    plot_items.append((best_sl, 'darkgreen', 'o-', 'SL'))

for r, color, marker, prefix in plot_items:
    ax.plot(range(N_WINDOWS), r['window_alphas'], marker, color=color,
            label=f"{prefix}: {r['name']}", markersize=6, linewidth=1.5)
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=1, alpha=0.7)
ax.set_xlabel('Window index', fontsize=12)
ax.set_ylabel(r'$\alpha$', fontsize=12)
ax.set_title('Shifted-log vs Polynomial vs 2D: Window Alpha Comparison', fontsize=13)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'theta_sl_vs_poly_vs_2d.png'), dpi=150, bbox_inches='tight')
plt.show()

## 10. Validation Scorecard

In [ ]:
# ================================================================
# VALIDATION SCORECARD
# ================================================================
print("\n" + "=" * 80)
print("VALIDATION SCORECARD")
print("=" * 80)

# Best candidate
best = phase1_ranked[0]
name = best['name']

print(f"\nCandidate: {name}")
if best['d_shift'] != 0:
    print(f"  theta(T) = {best['a']:.8f} - {best['b']:.8f}/(logT + ({best['d_shift']:.8f}))")
elif best['c'] != 0:
    print(f"  theta(T) = {best['a']:.8f} - {best['b']:.8f}/logT + {best['c']:.8f}/log^2 T")
else:
    print(f"  theta(T) = {best['a']:.8f} - {best['b']:.8f}/logT")
print(f"  Mode: {best['mode']}")
print()

# Collect all test results
tests = {
    'T5a (beat constant baselines)': T5a_pass,
    'T5b (beat random adaptive baselines)': T5b_pass,
    'T5 (combined)': T5_pass,
}
if name in t7_results:
    tests['T7 (bootstrap CI contains 1)'] = t7_results[name]['T7_pass']
if name in t8_results:
    tests['T8 (no drift, p > 0.05)'] = t8_results[name]['T8_pass']

n_pass = sum(1 for v in tests.values() if v)
n_total = len(tests)

print(f"  {'Test':<40} {'Result':>8}")
print(f"  {'-'*48}")
for test_name, passed in tests.items():
    status = 'PASS' if passed else 'FAIL'
    symbol = '+' if passed else 'X'
    print(f"  [{symbol}] {test_name:<37} {status:>8}")

print(f"\n  Score: {n_pass}/{n_total} tests passed")

# Detailed metrics
print(f"\n  Detailed metrics:")
print(f"    alpha(2M)       = {best['alpha']:+.6f}")
print(f"    |alpha - 1|     = {best['abs_alpha_minus_1']:.6f}")
print(f"    R^2             = {best['R2']:.6f}")
print(f"    R^2 (scaled)    = {best['R2_scaled']:.6f}")
print(f"    Localization    = {best['localization']:.4f}")
print(f"    Drift slope     = {best['drift_slope']:+.6f}  (p = {best['drift_p']:.4f})")
if name in t7_results:
    print(f"    Bootstrap CI    = [{t7_results[name]['ci_lo']:.6f}, {t7_results[name]['ci_hi']:.6f}]")
if name in t8_results:
    print(f"    Fine drift      = {t8_results[name]['drift_slope']:+.6f}  (p = {t8_results[name]['drift_p']:.4f})")

# Highlight if GIFT pure formula wins
if 'SL: 7/6 - phi/(logT - 2)' in name:
    print(f"\n  *** GIFT-PURE TOPOLOGICAL FORMULA ***")
    print(f"  All constants derived from topology:")
    print(f"    7/6 = dim(K7) / (2 * N_gen)")
    print(f"    phi = golden ratio (G2 metric eigenvalue)")
    print(f"    2   = p2 (Pontryagin class)")
    print(f"  Zero free parameters.")

## 11. Save Results

In [ ]:
# ================================================================
# SAVE RESULTS
# ================================================================

# Strip non-serializable data (delta_pred arrays)
save_results = []
for r in phase1_ranked:
    sr = {k: v for k, v in r.items() if k != 'delta_pred'}
    # Add T5, T7, T8 results
    if r['name'] == phase1_ranked[0]['name']:
        sr['T5a_pass'] = bool(T5a_pass)
        sr['T5a_margin'] = float(T5a_margin)
        sr['T5b_pass'] = bool(T5b_pass)
        sr['T5b_margin'] = float(T5b_margin)
        sr['T5_pass'] = bool(T5_pass)
    if r['name'] in t7_results:
        sr['T7_ci_lo'] = t7_results[r['name']]['ci_lo']
        sr['T7_ci_hi'] = t7_results[r['name']]['ci_hi']
    if r['name'] in t8_results:
        sr['T8_drift_slope_fine'] = t8_results[r['name']]['drift_slope']
        sr['T8_drift_p_fine'] = t8_results[r['name']]['drift_p']
    save_results.append(sr)

out_path = os.path.join(OUTPUT_DIR, 'theta_validation_results.json')
with open(out_path, 'w') as f:
    json.dump(save_results, f, indent=2)
print(f"Results saved to {out_path}")

# Summary
print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
best = phase1_ranked[0]
print(f"Best candidate: {best['name']} ({best['mode']})")
print(f"  alpha(2M) = {best['alpha']:+.6f}")
passed_tests = [k for k, v in tests.items() if v]
failed_tests = [k for k, v in tests.items() if not v]
if passed_tests:
    print(f"  Passed: {', '.join(passed_tests)}")
if failed_tests:
    print(f"  Failed: {', '.join(failed_tests)}")
print(f"\nAll {len(CANDIDATES)} candidates ranked in {out_path}")